In [18]:
from googleapiclient.discovery import build
import json
import os
import pandas as pd

In [19]:
current_dirt = os.getcwd()
apiKey_dirt = current_dirt + '\\APIkey.json'
channels_dirt = current_dirt + '\\channels.txt'

target_channels = []
with open(channels_dirt) as f:
    target_channels = f.read().splitlines()


with open(apiKey_dirt) as json_file:
    json_data = json.load(json_file)
    youtubeApiKey = json_data["APIkey"]

In [20]:
class youtuberCrawler:
    def __init__(self, channelId):
        self.corpus = []
        self.recent_videos = []
        self.popular_videos = []
        self.target_channel = channelId
        self.youtube = build('youtube','v3',developerKey=youtubeApiKey)

    def get_channelInfo(self):
        #채널 설명과 태그 긁어오기
        try:
            channels = self.youtube.channels().list(part=' brandingSettings', id=self.target_channel, fields='items(brandingSettings)').execute()
            self.corpus.append(channels['items'][0]['brandingSettings']['channel']['description'])
            self.corpus.append(channels['items'][0]['brandingSettings']['channel']['keywords'])
        
        except Exception as e:
            print(e)

    def get_recent_videoIds(self):
        #최근 10개 동영상 id 획득
        try:
            newpageToken = ''
            for i in range(1,3):
                activites = self.youtube.activities().list(part=' contentDetails', channelId=self.target_channel , fields='pageInfo, nextPageToken, items(contentDetails(upload(videoId)))', pageToken=newpageToken).execute()
                newpageToken = activites['nextPageToken']
                # print(newpageToken)
                for j in range(0,5):
                    self.recent_videos.append(activites['items'][j]['contentDetails']['upload']['videoId'])
        
        except Exception as e:
            print(e)
        
    def get_popular_videoIds(self):
        #조회수 탑 10 동영상 id 획득
        try:
            newpageToken = ''
            for i in range(1,3):
                search = self.youtube.search().list(part='snippet', channelId=self.target_channel, type='video', order='viewCount', fields='pageInfo, nextPageToken, items(id(videoId))').execute()
                newpageToken = search['nextPageToken']

                for j in range(0,5):
                    self.popular_videos.append(search['items'][j]['id']['videoId'])
                    
        except Exception as e:
            print(e)


    def get_video_info(self):
        try:
            #최근 동영상의 정보
            for i in range(0,10):
                videos = self.youtube.videos().list(part=' snippet', id=self.recent_videos[i], fields='items(snippet(title, description, tags) )').execute()
                self.corpus.append(videos['items'][0]['snippet'].get('title'))
                self.corpus.append(videos['items'][0]['snippet'].get('description'))
                tags_str = ' '.join(videos['items'][0]['snippet'].get('tags'))
                self.corpus.append(tags_str)
        
            #인기 동영상의 정보
            for i in range(0,10):
                videos = self.youtube.videos().list(part=' snippet', id=self.popular_videos[i], fields='items(snippet(title, description, tags) )').execute()
                self.corpus.append(videos['items'][0]['snippet'].get('title'))
                self.corpus.append(videos['items'][0]['snippet'].get('description'))
                tags_str = ' '.join(videos['items'][0]['snippet'].get('tags'))
                self.corpus.append(tags_str)
                
        except Exception as e:
            print(e)

            
    def result_str_Corpus(self):
        try:
            result_Str = "\n".join(self.corpus)
            return result_Str
        
        except Exception as e:
            print(e)

In [21]:
result_dirt = current_dirt + '\\crawling_result\\youtuber_text'

In [24]:
# 유튜버 데이터 저장하기
def get_youtuber_info(channelId):
    print("get info of ", channelId)
    f = open(result_dirt + '\\' + channelId + ".txt", mode='wt', encoding='utf-8')
    crawler = youtuberCrawler(channelId)
    crawler.get_channelInfo()
    crawler.get_recent_videoIds()
    crawler.get_popular_videoIds()
    crawler.get_video_info()
    f.write(crawler.result_str_Corpus())
    f.close()

In [25]:
for i in target_channels:
    get_youtuber_info(i)

get info of  UCbFzvzDu17eDZ3RIeaLRswQ
get info of  UCUbOogiD-4PKDqaJfSOTC0g
get info of  UCVJT18d9wSCnDUdnJ9ycO7Q
get info of  UCnb4EuP3C0dxeika3b-bnXA
can only join an iterable
get info of  UChQ-VMvdGrYZxviQVMTJOHg
'upload'
list index out of range
get info of  UCg7rkxrTnIhiHEpXY1ec9NA
'upload'
list index out of range
get info of  UCNzcxCN_Hh_lu5RCSFXKgGQ
'keywords'


KeyboardInterrupt: 